<a href="https://colab.research.google.com/github/podyssea/InformationRetrieval/blob/main/InformationRetrievalExercise2_2210049p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Exercise 2 Notebook 


This is the template notebook for Exercise 2. The specification for the exercise and the corresponding Exercise 2 submission instance are available on the Moodle page of the course.

This exercise builds upon Exercise 1, and assumes you are now familiar with concepts we have introduced in both the introductory lab and Exercise 1, including:
 - [PyTerrier operators](https://pyterrier.readthedocs.io/en/latest/operators.html)
 - [Pyterrier apply transformers](https://pyterrier.readthedocs.io/en/latest/transformer.html)
 - [PyTerrier pt.Experiment()](https://pyterrier.readthedocs.io/en/latest/experiments.html)


## PyTerrier Setup

Firt, we need to install PyTerrier. If you have been using your own machine, you will need to upgrade PyTerrier to the latest version for running this Exercise 2 notebook.

In [ ]:
!date
!pip install python-terrier


Sat Feb 27 00:08:42 UTC 2021


Start PyTerrier; This time we are specifying `tqdm='notebook'` to get pretty progress bars during retrieval. 

In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init(tqdm='notebook')

## Index, Topics & Qrels for Exercise 2

You will need your login & password credentials from Exercise 1. We will be using again the "50pct" and the "trec-wt-2004" datasets from Exercise 1.


In [ ]:
UNAME="TODO"
PWORD="TODO"

from pyterrier.datasets import STANDARD_TERRIER_INDEX_FILES, RemoteDataset

# we will again be using the "50pct" and "trec-wt-2004" datasets
Fiftypct =  pt.get_dataset("50pct",  user="2210049p", password="e54de6e4")
dotgov_topicsqrels = pt.get_dataset("trec-wt-2004")

On the other hand, you will be using a slightly updated index for Exercise 2. It is a bit bigger than the Exercise 1 index, hence it takes about 2-3 minutes to download to Colab.

In [ ]:
!date
indexref = Fiftypct.get_index(variant="ex2")
index = pt.IndexFactory.of(indexref)
!date

Sat Feb 27 00:09:03 UTC 2021
00:09:03.603 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 860.9 MiB of memory would be required.
Sat Feb 27 00:09:03 UTC 2021


Let's check out the new index. Compared to the index we used for Exercise 1, you can see that this index has `Field Names: [TITLE, ELSE]`, which means that we can provide statistics about how many times each term occurs in the title of each document (the "TITLE" field), vs the rest of the document (the "ELSE" field). Refer to Lecture 8 for more information about fields.

Let's also display the keys in the meta index - this is the metadata that we have stored for each document. You can see that we are storing the "url" and the "body" (content) of the document. These will particularly come in handy for Q2 and Q3 of Exercise 2, respectively.


In [ ]:
print(index.getCollectionStatistics().toString())
print("In the meta index: " + str(index.getMetaIndex().getKeys()))

Number of documents: 807775
Number of terms: 2043788
Number of postings: 177737957
Number of fields: 2
Number of tokens: 572916194
Field names: [TITLE, ELSE]
Positions:   true

In the meta index: ['docno', 'url', 'title', 'body']


Finally, these are all of the topics and qrels (including the training and validation datasets) that you will need to conduct Exercise 2.

In [ ]:
# index.getMetaIndex().getItem("url", 513586)

In [ ]:
tr_topics = Fiftypct.get_topics("training")
va_topics = Fiftypct.get_topics("validation")

tr_qrels = Fiftypct.get_qrels("training")
va_qrels = Fiftypct.get_qrels("validation")

test_topics = dotgov_topicsqrels.get_topics("hp")
test_qrels = dotgov_topicsqrels.get_qrels("hp")

## Baseline Setup

We introduce here the BatchRetrieve for our baseline. Note that:
 - We are using PL2 as our weighting model to generate the sample (the candidate set of documents to re-rank).
 - We expose more document metadata, namely "url" and "body" for each document retrieved, which you will need to deploy your two new features. 
 - By setting `verbose=True`, we display a progress bar while retrieval executes.

In [ ]:
firstpassUB = pt.BatchRetrieve(index, wmodel="PL2", metadata=["docno", "url", "body"], verbose=True)


Let's see the resulting output - you can see that there are now "url" and "body" attributed for each retrieved document. (We also display a progress bar, enabled by the `verbose=True`).

In [ ]:
firstpassUB.search("chemical reactions")

,qid,docid,docno,url,body,rank,score,query
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. ...,1,11.906524,chemical reactions
2,1,707122,G26-06-3754605,http://www.aps.anl.gov/xfd/tech/safetyenvelope...,APS Experiment Safety Envelope 6: Chemicals ...,2,11.877550,chemical reactions
3,1,382754,G13-59-3981168,http://response.restoration.noaa.gov/chemaids/...,"""); } else { document.write(...",3,11.858475,chemical reactions
4,1,70292,G02-16-2617043,http://www.symp14.nist.gov/PDF/COR04MAY.PDF,A Database of Chemical Reactions Designed to A...,4,11.731490,chemical reactions
...,...,...,...,...,...,...,...,...
995,1,246965,G08-68-4141101,http://en-env.llnl.gov/asd/pinatub.html,The Chemical and Radiative Effects of the Moun...,995,6.290707,chemical reactions
996,1,611136,G22-04-3955177,http://eospso.gsfc.nasa.gov/ftp_docs/Ch7.pdf,Chapter 7 ...,996,6.289822,chemical reactions
997,1,594957,G21-38-0191596,http://www.oit.doe.gov/news/oittimes/wn02/wn02...,search ...,997,6.287830,chemical reactions
998,1,280944,G09-85-3411646,http://www.ig.doe.gov/pdf/chemfina.pdf,INS-O-00-01 I N S P E C T I O N ...,998,6.287759,chemical reactions


# Standard list of features

Let's introduce the list of features we need to deploy a baseline learning-to-rank approach.

In [ ]:
pagerankfile = indexref.toString().replace(".properties", "-pagerank.oos")
features = [
    "SAMPLE", #ie PL2
    "WMODEL:SingleFieldModel(BM25,0)",
    "QI:StaticFeature(OIS,%s)" % pagerankfile,
]

stdfeatures = pt.FeaturesBatchRetrieve(index, features, verbose=True)
stage12 = firstpassUB >> stdfeatures

This is our feature set. We will be using FeaturesBatchRetrieve to compute these extra features on the fly. Let's see the output. You can see that there is now a "features" column.

In [ ]:
stage12.search("chemical reactions").head(2)

,qid,query,docid,rank,features,docno,score
0,1,chemical reactions,513586,0,"[12.755545561073266, 3.0924078763629836, 0.000...",G18-38-1767991,12.755546
1,1,chemical reactions,38544,1,"[11.90652405775751, 10.789390732195702, 0.0002...",G01-14-2537005,11.906524


Let's look in more detail at the features. It is clear that there are 3 numbers for each document. The first is the PL2 score (1.27555456e+01 == 12.7555), the second is the BM25 score, and the third is the PageRank (a link analysis feature - discussed in more detail in Lecture 10)

In [ ]:
stage12.search("chemical reactions").head(1).iloc[0]["features"]

array([1.27555456e+01, 3.09240788e+00, 1.05668333e-04])

# Q1

You now have everyting you need to attempt Q1. You will need to refer to the specification, and to PyTerrier's [learning to rank documentation](https://pyterrier.readthedocs.io/en/latest/ltr.html).

You should use a LightGBM LambdaMART implementation (not XGBoost), instantiated using the configuration suggested in the PyTerrier documentation.

Hints:
 - You will need to use the provided separate “training” and “validation” topic sets and qrels to train the learning-to-rank.
 - There is no need to vary the configuration of LightGBM from that in the documentation.

In [ ]:
#stage12 = firstpassUB >> stdfeatures

In [ ]:
import lightgbm as lgb
lmart_l = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)

In [ ]:
lmart_l_pipe = stage12 >> pt.ltr.apply_learned_model(lmart_l, form="ltr")

In [ ]:
lmart_l_pipe.fit(tr_topics, tr_qrels, va_topics, va_qrels)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.277778
[2]	valid_0's ndcg@1: 0.351852
[3]	valid_0's ndcg@1: 0.351852
[4]	valid_0's ndcg@1: 0.388889
[5]	valid_0's ndcg@1: 0.388889
[6]	valid_0's ndcg@1: 0.407407
[7]	valid_0's ndcg@1: 0.407407
[8]	valid_0's ndcg@1: 0.407407
[9]	valid_0's ndcg@1: 0.388889
[10]	valid_0's ndcg@1: 0.388889


In [ ]:
pt.Experiment([stage12, lmart_l_pipe], test_topics, test_qrels, ["map"], names=["PL2 Baseline", "LTR (PL2 Sample)"], baseline = 0)

,name,map,map +,map -,map p-value
0,PL2 Baseline,0.225110,NaN,NaN,NaN
1,LTR (PL2 Sample),0.410703,45.0,15.0,0.000097


In [ ]:
pt.Experiment([stage12, lmart_l_pipe], test_topics, test_qrels, ["P.5"], names=["PL2 Baseline", "LTR (PL2 Sample)"], baseline = 0)

,name,P_5,P_5 +,P_5 -,P_5 p-value
0,PL2 Baseline,0.069333,NaN,NaN,NaN
1,LTR (PL2 Sample),0.114667,22.0,6.0,0.001787


#Q2 - URL Length Features

In this block, please provide your code for Q2 concerning the URL length feature(s). There are different possible URL length features that you could implement (see specification) - you might try more than one implementation, and choose the one that is the most effective.

Some hints:

 - You will need to use a [pt.apply function](https://pyterrier.readthedocs.io/en/latest/apply.html) for computing your URL length feature(s). The dataframe of results obtained from the upstream transformer has all of the information you need.

 - You can use a `**` operator for combining feature sets.

 - Refer to the PyTerrier learning to rank documentation  concerning `features_importances_` for obtaining feature importances, as required in Q2(c).

 - You may wish to refer to Python's [`urlparse()`](https://docs.python.org/3/library/urllib.parse.html) function.

 - Use Python assertions to test that your feature implmentation(s) give the expected results. This is needed for Q2(b).


In [ ]:
def _url_feature(row):
  docid = row["docid"]
  url = index.getMetaIndex().getItem("url", docid)
  f1 = len(url)
  return f1

In [ ]:
url_length_boost = stage12 >> (pt.ltr.apply_learned_model(lmart_l, form="ltr") ** pt.apply.doc_features(_url_feature))

In [ ]:
import numpy as np
pt.Experiment([stage12, lmart_l_pipe, url_length_boost], test_topics, test_qrels, ["map"], names=["PL2 Baseline", "LTR (PL2 Sample)", "LTR (PL2 Sample) + URL Lengths"], baseline = 0)

ValueError: ignored

#Q3 Proximity Search Feature

Now you will implement a new query-dependent feature, using the MinDist() function, as discussed in the specification. 

Hints:
 - Again, remember to use assertions to test your feature implementations. This will be needed for Q3(b)
 - Refer to the PyTerrier learning to rank documentation concerning features_importances_ for obtaining feature importances, as required in Q3(c)



#Q4 A 5-feature Learning-to-Rank Model

You will now experiment with the LightGMB LambdaMART technique where you include both your added features (URL Length and AggMinDist) along the 3 initial features inc PL2 sample (5 features in total). 

You need to learn a *new* model when using your final selection of 5 features.

#Q5-Q6 Analysis & Reflection

The specification requests more analysis, including figures. Use more code here as needed to draw the graphs and prepare the data input needed for the table. For example, are there any patterns in the queries that are improved (or not) when adding your two deployed features? 

Once you have conducted your analysis, the specification asks you to reflect on what you have learnt from conducting the experiments and what your main conclusions are.

# That's all Folks

**Submission Instructions:** Complete this notebook. Please make your submission on the Moodle Exercise 2 submission instance. As part of your submission, you will need to hand-in the following:
- A **PDF report** (6 pages MAX) with your answers to questions Q1--Q6 of the specification
- Your **completed .ipynb Colab notbook** (use File... Download .ipynb).

Your PDF report along your ipynb notebook solution must be submitted by **Wednesday 24th March 2021, 4:30pm**.